# RESTful API Bitkub

In [1]:
import csv
import json
import time
import hmac
import random
import hashlib
import requests
import numpy as np
import pandas as pd
from datetime import datetime

In [50]:
class bitkub():
    
    def __init__(self, API_HOST, API_KEY, API_SECRET, line_token=None, symbol=None):
        self.API_HOST = API_HOST
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')
        self.header = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'X-BTK-APIKEY': API_KEY,
        }
        self.symbol = symbol
        self.token = line_token
        
    def json_encode(self, data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)

    def sign(self, data):
        j = self.json_encode(data)
        #print('Signing payload: ' + j)
        h = hmac.new(self.API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
        return h.hexdigest()

    def timeserver(self):
        response = requests.get(self.API_HOST + '/api/servertime')
        ts = int(response.text)
        return ts
        
    def allSymbol(self):
        response = requests.get(self.API_HOST + '/api/market/symbols')
        symbols = pd.DataFrame(response.json()['result'])
        return symbols
        
    def getPrice(self):
        pair = self.symbol
        response = requests.get(self.API_HOST + '/api/market/ticker')
        price = float(response.json()[pair]['last'])
        return price
    
    def createBuy(self, amount, rate, typ):
        data = {
            'sym': self.symbol, # THB_XLM
            'amt': amount, # THB amount you want to spend 
            'rat': rate, # Price
            'typ': typ, # limit Order
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/place-bid', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 0:
            print(f'Buy {typ}: {self.symbol} @{rate} amount: {amount} THB')
            
        elif r['error'] == 18:
            print(f'Error 18: Insufficient balance.')
        
        elif r['error'] == 15:
            print('Error 15: Amount too low.')
        
        else:
            print('Error: Please check the code.')

    
    def createSell(self, amount, rate, typ):
        data = {
            'sym': self.symbol, # THB_XLM
            'amt': amount, # XLM unit you want to sell 
            'rat': rate, # Price
            'typ': typ, # limit Order
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/place-ask', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 0:
            print(f'Sell {typ}: {self.symbol} @{rate} amount: {amount} unit')
            
        elif r['error'] == 18:
            print(f'Error 18: Insufficient balance.')
        
        elif r['error'] == 15:
            print('Error 15: Amount too low.')
        
        else:
            print('Error: Please check the code.')
        
        
    def my_open_orders(self):
        data = {
            'sym': self.symbol,
            'ts' : self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/my-open-orders', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_r =  pd.DataFrame(r, columns=['id', 'side', 'type', 'rate', 'fee', 'amount', 'receive'])
        return df_r 
        
    def cancelOrder(self, order_id, side):
        data = {
            'sym': self.symbol,
            'id': order_id,
            'sd': side,
            'ts' : self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/cancel-order', headers=self.header, data=self.json_encode(data))
        r = r.json()
        
        if r['error'] == 21:
            print(f'Order ID: {order_id} Invalid order for cancellation.')
            
        elif r['error'] == 0:
            print(f'Order ID: {order_id} has been cancelled.')
        
        else:
            print('Cancel error please check the code.')
            
            
    def wallet(self): # Get user available balances
        data = {
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/wallet', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        return r
    
    def balances(self): 
        # Includes both available and reserved balances
        # coin = 'THB', 'XLM'
        data = {
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/balances', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_total = pd.DataFrame.from_dict(r) # .sum(axis=0)
        return df_total
    
    def order_history(self):
        data = {
            'sym': self.symbol,
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/my-order-history', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_r = pd.DataFrame(r)
        return df_r
    
    def cancelAllOrder(self):
        order_id = self.my_open_orders()['id']
        side = self.my_open_orders()['side']
        for i,j in zip(order_id,side):
            self.cancelOrder(i, j)
    
    def order_info(self, order_id, side):
        # For check status
        data = {
            'sym': self.symbol,
            'id': order_id,
            'sd': side,
            'ts': self.timeserver(),
        }
        
        signature = self.sign(data)
        data['sig'] = signature
        r = requests.post(self.API_HOST + '/api/market/order-info', headers=self.header, data=self.json_encode(data))
        r = r.json()['result']
        df_r = pd.DataFrame(r)
        return df_r
    
    def getPriceHistory(self, tf, frm=None, to=None):
        params = {
            'sym': self.symbol,
            'int': tf, # 60 900 3600 86400
            'frm': frm,
            'to': to,
        }
        
        signature = self.sign(params)
        params['sig'] = signature
        
        #'sym=THB_XLM&lmt=1'
        r = requests.get(self.API_HOST + '/api/market/tradingview', params=self.json_encode(params))
        r = r.json()['result']
        df_r = pd.DataFrame(r)
        return df_r
        
               
# fetch price history

In [51]:
bk = bitkub(API_HOST='https://api.bitkub.com',
            API_KEY='15532eb7e3a3934d19ce85a121c38e85', 
            API_SECRET='ec3a0a80a1e1c69ada837dc89aa4af85',
            symbol='THB_XLM')

# calculate cashflow

In [136]:
class calulateLIFO():
    
    def __init__(self, data): # 'side', 'price, 'amount(unit)'
        self.orderhis = data
        self.lifo_price = []
        self.lifo_unit = []
        self.cashflow = 0

    def buyinLIFO(self, buy_unit, buy_price):
        self.lifo_price.append(buy_price)
        self.lifo_unit.append(buy_unit)
        
      
    def soldoutLIFO(self, sold_unit, sold_price):
    
        sum_unit = 0
        list_unit = []
        list_price = []
          
        while  sum_unit < sold_unit:
            u = self.lifo_unit.pop()
            p = self.lifo_price.pop()
            sum_unit += u
            list_unit.append(u)
            list_price.append(p)
    
        for u, p in zip(list_unit, list_price):
            if u>sold_unit:
                self.cashflow += sold_unit*(sold_price-p) 
                self.lifo_unit.append(u-sold_unit)
                self.lifo_price.append(p)
            else:
                self.cashflow += u*(sold_price-p) 
                sold_unit-=u
   

    def cashflowLIFO(self):
        order_his = self.orderhis
    
        for i in range(len(order_his)):
            if order_his['side'][i] == 'buy':
                self.buyinLIFO(order_his['amount'][i], order_his['rate'][i])
            
            elif order_his['side'][i] == 'sell':
                self.soldoutLIFO(order_his['amount'][i], order_his['rate'][i])
                

In [142]:
df = bk.order_history()[['side', 'rate', 'amount']].iloc[:3]
df = df.iloc[::-1]
df = df.reset_index(drop=True)

In [143]:
df

,side,rate,amount
0,buy,11.50,34.782609
1,buy,10.75,2.418605
2,sell,11.20,1.486607


In [144]:
calCF = calulateLIFO(df)

In [145]:
calCF.cashflowLIFO()

In [146]:
calCF.cashflow

0.6689732174999989